# Collaborative filtering

<a id='0'></a>

## Table of contents
[1.Matrix factorization](#1)   
[2.TF-IDF](#2)

In [3]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

<a id='1'></a>

## [Matrix factorization](#0)

In [4]:
# create example data for collaborative filtering
UserItemMatrix = np.array([np.array([5, np.nan, 4, np.nan, 1, np.nan, 3]),
                           np.array([4, 4, 4, np.nan, np.nan, np.nan, 1]),
                           np.array([5, 4, np.nan, 1, 2, np.nan, 3]),
                           np.array([1, 2, 1, 4, 3, 5, 2]),
                           np.array([np.nan, 1, np.nan, 3, 5, 5, np.nan]),
                           np.array([np.nan, 2, np.nan, np.nan, 4, 4, 2]),
                           np.array([5, np.nan, np.nan, 1, np.nan, np.nan, 2])
                          ])
UserItemMatrix

array([[ 5., nan,  4., nan,  1., nan,  3.],
       [ 4.,  4.,  4., nan, nan, nan,  1.],
       [ 5.,  4., nan,  1.,  2., nan,  3.],
       [ 1.,  2.,  1.,  4.,  3.,  5.,  2.],
       [nan,  1., nan,  3.,  5.,  5., nan],
       [nan,  2., nan, nan,  4.,  4.,  2.],
       [ 5., nan, nan,  1., nan, nan,  2.]])

In [5]:
# explore User-Item matrix
df = pd.DataFrame(UserItemMatrix, 
                  columns=['item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'item7'])
df['user_id'] = list(df.index)
df.head()

,item1,item2,item3,item4,item5,item6,item7,user_id
0,5.0,NaN,4.0,NaN,1.0,NaN,3.0,0
1,4.0,4.0,4.0,NaN,NaN,NaN,1.0,1
2,5.0,4.0,NaN,1.0,2.0,NaN,3.0,2
3,1.0,2.0,1.0,4.0,3.0,5.0,2.0,3
4,NaN,1.0,NaN,3.0,5.0,5.0,NaN,4


In [7]:
!pip install torch

In [8]:
# R: 복원하고자 하는 matrix
# K: latent dimension
# 참고: pytorch의 Tensor로 SGD 알고리즘을 적용해서 matrix factorization이 가능함.
import torch

def matrix_factorization_using_torch(R, K, steps=200, learning_rate=0.01, set_seed = True, verbose = True, smooth= True):
    
    num_users, num_items = R.shape
    
    if set_seed:
        np.random.seed(1)
    
    # initailize user-latent, item-latent matrices.
    
    P = torch.randn((num_users, K),requires_grad=True)
    Q = torch.randn((num_items, K),requires_grad=True)
    
    # R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장한다.
    R_copy = R.copy()
    R_copy[np.isnan(R_copy)] = 0
    non_zeros_idx = torch.tensor(R_copy > 0)
    R_copy = torch.tensor(R_copy)

    # SGD 기법으로 P, Q 매트릭스를 업데이트 함
    optimizer = torch.optim.SGD([P,Q], lr= learning_rate, momentum= 0.9)
    
    for step in range(steps):
        pred = P@Q.T
        loss = torch.mean((R_copy[non_zeros_idx] - pred[non_zeros_idx])**2)
        
        with torch.no_grad():
            rmse = torch.sqrt(loss)
        if step % 10 == 0 and verbose:
            print("iter step: {0}, rmse: {1:4f}".format(step, rmse))
            
        # zero gradients, perform a backward pass, and update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    R_hat = P.detach().numpy()@Q.detach().numpy().T
    
    if smooth:
        R_hat_round = np.round(R_hat)
        R_hat_round[R_hat_round>5]= 5
        R_hat_round[R_hat_round<1]= 1
        R_hat = R_hat_round
    
    return P.detach().numpy(), Q.detach().numpy(), R_hat


In [11]:
P, Q, R_hat = matrix_factorization_using_torch(UserItemMatrix,3)
#왼쪽,오른쪽, 추정값

iter step: 0, rmse: 3.827588
iter step: 10, rmse: 3.307323
iter step: 20, rmse: 2.821157
iter step: 30, rmse: 2.226036
iter step: 40, rmse: 1.375080
iter step: 50, rmse: 0.803308
iter step: 60, rmse: 0.589498
iter step: 70, rmse: 0.493004
iter step: 80, rmse: 0.431299
iter step: 90, rmse: 0.385970
iter step: 100, rmse: 0.370660
iter step: 110, rmse: 0.359873
iter step: 120, rmse: 0.351398
iter step: 130, rmse: 0.345203
iter step: 140, rmse: 0.339479
iter step: 150, rmse: 0.334178
iter step: 160, rmse: 0.329126
iter step: 170, rmse: 0.324209
iter step: 180, rmse: 0.319391
iter step: 190, rmse: 0.314642


In [12]:
# User matrix
UserItemMatrix

array([[ 5., nan,  4., nan,  1., nan,  3.],
       [ 4.,  4.,  4., nan, nan, nan,  1.],
       [ 5.,  4., nan,  1.,  2., nan,  3.],
       [ 1.,  2.,  1.,  4.,  3.,  5.,  2.],
       [nan,  1., nan,  3.,  5.,  5., nan],
       [nan,  2., nan, nan,  4.,  4.,  2.],
       [ 5., nan, nan,  1., nan, nan,  2.]])

In [14]:
# Prediction
R_hat

array([[5., 5., 4., 3., 1., 2., 3.],
       [4., 4., 4., 1., 1., 1., 1.],
       [5., 4., 1., 1., 2., 2., 3.],
       [1., 2., 1., 3., 4., 5., 2.],
       [1., 1., 1., 4., 4., 5., 2.],
       [3., 2., 1., 2., 4., 4., 2.],
       [5., 3., 1., 1., 2., 2., 2.]], dtype=float32)

In [15]:
# explore User-Item matrix
df_filled = pd.DataFrame(R_hat, 
                  columns=['item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'item7'])
df_filled['user_id'] = list(df_filled.index)
df_filled.head()

,item1,item2,item3,item4,item5,item6,item7,user_id
0,5.0,5.0,4.0,3.0,1.0,2.0,3.0,0
1,4.0,4.0,4.0,1.0,1.0,1.0,1.0,1
2,5.0,4.0,1.0,1.0,2.0,2.0,3.0,2
3,1.0,2.0,1.0,3.0,4.0,5.0,2.0,3
4,1.0,1.0,1.0,4.0,4.0,5.0,2.0,4


In [16]:
# please compare the result with the raw data: df
df.head()

,item1,item2,item3,item4,item5,item6,item7,user_id
0,5.0,NaN,4.0,NaN,1.0,NaN,3.0,0
1,4.0,4.0,4.0,NaN,NaN,NaN,1.0,1
2,5.0,4.0,NaN,1.0,2.0,NaN,3.0,2
3,1.0,2.0,1.0,4.0,3.0,5.0,2.0,3
4,NaN,1.0,NaN,3.0,5.0,5.0,NaN,4


## 실제 데이터 활용

* 전처리 방식에 주목해주세요.

In [17]:
rating_data, movie_data = pd.read_csv('./Data/ratings.csv'), pd.read_csv('./Data/movies.csv')

In [18]:
rating_data.drop('timestamp',axis=1,inplace=True)
rating_data.head(3)

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0


In [19]:
movie_data.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [20]:
# movieid 를 기준으로 두 데이터 프레임을 합침
user_movie_rating = pd.merge(rating_data, movie_data, on='movieId')

In [21]:
user_movie_rating.head()

,userId,movieId,rating,title,genres
0,1,31,2.5,Dangerous Minds (1995),Drama
1,7,31,3.0,Dangerous Minds (1995),Drama
2,31,31,4.0,Dangerous Minds (1995),Drama
3,32,31,4.0,Dangerous Minds (1995),Drama
4,36,31,3.0,Dangerous Minds (1995),Drama


* 사용자 - 영화 평점 점수 데이터 형식으로 바꿔줘야 합니다.

In [22]:
user_movie_rating = user_movie_rating.pivot_table('rating', index= 'userId',columns='title')

In [23]:
user_movie_rating.head()

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
R = user_movie_rating.values

In [25]:
# Note: K(latent dim) is the hyper-parameter to be tuned.
P, Q, R_hat= matrix_factorization_using_torch(R, 1000, smooth= False)

iter step: 0, rmse: 31.718379
iter step: 10, rmse: 31.053504
iter step: 20, rmse: 29.815210
iter step: 30, rmse: 28.444471
iter step: 40, rmse: 27.095669
iter step: 50, rmse: 25.817241
iter step: 60, rmse: 24.619844
iter step: 70, rmse: 23.501423
iter step: 80, rmse: 22.456200
iter step: 90, rmse: 21.477787
iter step: 100, rmse: 20.560169
iter step: 110, rmse: 19.697961
iter step: 120, rmse: 18.886408
iter step: 130, rmse: 18.121319
iter step: 140, rmse: 17.398988
iter step: 150, rmse: 16.716127
iter step: 160, rmse: 16.069800
iter step: 170, rmse: 15.457376
iter step: 180, rmse: 14.876483
iter step: 190, rmse: 14.324979


<a id='2'></a>

## [TF-IDF](#0)

TF-IDF:
Term Frequency - Inverse Document Frequency (직역: 단어의 빈도와 역 문서 빈도)   
__특정 문서 내에 있는 어떤 단어가 얼마나 중요한 지를 나타내는 통계량__입니다.   
basic idea: 특정 문서에 특별히 자주 등장하는 단어가 해당 문서의 키워드라는 전제가 깔려있습니다.   

* TF (Term Frequency): 문서 내 특정 단어의 빈도를 의미합니다.    
* IDF (Inverse Document Frequency): DF (Document frequency) 의 역수이며, 특정 단어가 발견되는 문서의 수입니다.     
(참고로 특정 단어가 IDF가 크다는 것은 대부분의 문서에서 발견된다는 뜻입니다. 즉, 그 단어가 덜 중요하다고 해석할 수 있습니다.) 
* TF-IDF : TF $\times$ IDF 

주로 문서의 유사도를 구하는 작업 또는 특정 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있습니다.


In [26]:
# 사이킷런은 TF-IDF를 자동 계산해주는 TfidfVectorizer를 제공합니다.
from sklearn.feature_extraction.text import TfidfVectorizer
documents = [
    'attention is all you need',
    'you have my word',
    'I like you',
    'what you should do ',
    'you you you why you',
    'you, you, you, you everywhere'
]
tfidfv = TfidfVectorizer().fit(documents)
results = tfidfv.transform(documents).toarray()
vocab = tfidfv.vocabulary_
print(results)
print(vocab)

[[0.48812169 0.48812169 0.         0.         0.         0.48812169
  0.         0.         0.48812169 0.         0.         0.
  0.         0.2166769 ]
 [0.         0.         0.         0.         0.55927514 0.
  0.         0.55927514 0.         0.         0.         0.
  0.55927514 0.24826187]
 [0.         0.         0.         0.         0.         0.
  0.91399636 0.         0.         0.         0.         0.
  0.         0.40572238]
 [0.         0.         0.55927514 0.         0.         0.
  0.         0.         0.         0.55927514 0.55927514 0.
  0.         0.24826187]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.49071837
  0.         0.87131824]
 [0.         0.         0.         0.49071837 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.87131824]]
{'attention': 1, 'is': 5, 'all': 0, 'you': 13, 'need': 8, 'have': 4, 'my': 7, 'word': 12, 'like': 6, 'what': 10, 

In [27]:
# 결과를 좀 더 직관적으로 볼 수 있게 정리했습니다.
vocab_sorted = sorted(vocab.items(), key=lambda x: x[1], reverse=False)
features = [k for k,v in vocab_sorted]
df = pd.DataFrame(results)
df.columns =features
df['words'] = documents
df.set_index('words',inplace=True)

In [28]:
df.head(6)

,all,attention,do,everywhere,have,is,like,my,need,should,what,why,word,you
words,,,,,,,,,,,,,,
attention is all you need,0.488122,0.488122,0.000000,0.000000,0.000000,0.488122,0.000000,0.000000,0.488122,0.000000,0.000000,0.000000,0.000000,0.216677
you have my word,0.000000,0.000000,0.000000,0.000000,0.559275,0.000000,0.000000,0.559275,0.000000,0.000000,0.000000,0.000000,0.559275,0.248262
I like you,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.913996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.405722
what you should do,0.000000,0.000000,0.559275,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.559275,0.559275,0.000000,0.000000,0.248262
you you you why you,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.490718,0.000000,0.871318
"you, you, you, you everywhere",0.000000,0.000000,0.000000,0.490718,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.871318
